In [1]:
%reload_ext autoreload
%autoreload 2

import h5py
N_Cs = 9

with h5py.File('../dxtb/dxtb-gpu/gpu-cpu_analysis/rdkit/alkanes_data_500.hdf5', 'r') as f:
    for mol_name, data in f.items():
        if mol_name == f"alkane_{N_Cs}_carbons":
            atomic_numbers = data['atomic_numbers'][:]
            coordinates = data['coordinates'][:]

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")

Number of carbon atoms in alkane_9_carbons: 9
Nb of atoms: 29


In [5]:
import dxtb
from dxtb._src.typing import DD
import torch
from ase.build import molecule
from dxtb.config import ConfigCache

opts = {"scf_mode": "implicit", "batch_mode": 2, "int_driver": "libcint", "maxiter": 1}
batch_size = 64
results = {}

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")
print(f"batch_size: {batch_size}")
print(f"opts: {opts}")

for device in ["cuda:0", "cpu"]:
    print(f"\nDevice: {device}")
    dd = {"dtype": torch.float32, "device": torch.device(device)}
    numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
    positions = torch.tensor(coordinates, device=dd["device"], dtype=dd["dtype"])
    numbers = torch.stack([numbers] * batch_size)
    positions = torch.stack([positions] * batch_size).requires_grad_()
    charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

    calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
    calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)

    
    dxtb.timer.reset()
    e = calc.get_energy(positions, chrg=charges)
    dxtb.timer.start("Forces autograd")
    forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
    dxtb.timer.stop("Forces autograd")
    dxtb.timer.print(v=0)

    # Store drv
    drv = calc.integrals.mgr.driver.drv
    print(f"len(drv): {len(drv)}")

    results[device] = {
        "energy": e.detach().cpu(),
        "forces": forces.detach().cpu()
    }

# Compare results
energy_diff = (results["cuda:0"]["energy"] - results["cpu"]["energy"]).abs().max()
forces_diff = (results["cuda:0"]["forces"
] - results["cpu"]["forces"]).abs().max()

print(f"\n[Comparison]")
print(f"GPU energy: {results['cuda:0']['energy'].mean().item():.6e}")
print(f"CPU energy: {results['cpu']['energy'].mean().item():.6e}")
print(f"Max energy diff: {energy_diff.item():.6e}")
print(f"Max forces diff: {forces_diff.item():.6e}")

Number of carbon atoms in alkane_9_carbons: 9
Nb of atoms: 29
batch_size: 64
opts: {'scf_mode': 'implicit', 'batch_mode': 2, 'int_driver': 'libcint', 'maxiter': 1}

Device: cuda:0


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.073          11.23
 - Halogen                  0.042          57.27
 - DispersionD3             0.016          22.50
 - Repulsion                0.015          20.01
Integrals                   0.223          34.45
 - Overlap                  0.205          92.28
 - Core Hamiltonian         0.017           7.71
SCF                         0.208          32.15
 - Interaction Cache        0.004           2.14
 - Potential                0.159          76.65
 - Fock build               0.007           3.38
 - Diagonalize              0.131          62.91
 - Density                  0.002           1.05
 - Charges                  0.008           4.08
Forces autograd  

In [6]:
import dxtb
from dxtb._src.typing import DD
import torch
from ase.build import molecule
from dxtb.config import ConfigCache


for device in ["cuda:0", "cpu"]:
    print(f"\nDevice: {device}")
    dd = {"dtype": torch.float32, "device": torch.device(device)}
    numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
    positions = torch.tensor(coordinates, device=dd["device"], dtype=dd["dtype"])
    numbers = torch.stack([numbers] * batch_size)
    positions = torch.stack([positions] * batch_size).requires_grad_()
    charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

    calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
    calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)

    
    dxtb.timer.reset()
    e = calc.get_energy(positions, chrg=charges, drv=drv)
    dxtb.timer.start("Forces autograd")
    forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
    dxtb.timer.stop("Forces autograd")
    dxtb.timer.print(v=0)

    results[device] = {
        "energy": e.detach().cpu(),
        "forces": forces.detach().cpu()
    }

# Compare results
energy_diff = (results["cuda:0"]["energy"] - results["cpu"]["energy"]).abs().max()
forces_diff = (results["cuda:0"]["forces"
] - results["cpu"]["forces"]).abs().max()

print(f"\n[Comparison]")
print(f"GPU energy: {results['cuda:0']['energy'].mean().item():.6e}")
print(f"CPU energy: {results['cpu']['energy'].mean().item():.6e}")
print(f"Max energy diff: {energy_diff.item():.6e}")
print(f"Max forces diff: {forces_diff.item():.6e}")


Device: cuda:0


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.049          10.51
 - Repulsion                0.003           5.55
 - Halogen                  0.023          46.65
 - DispersionD3             0.023          47.54
Integrals                   0.057          12.28
 - Overlap                  0.048          85.03
 - Core Hamiltonian         0.008          14.92
SCF                         0.194          41.86
 - Interaction Cache        0.012           5.94
 - Potential                0.129          66.64
 - Fock build               0.002           0.94
 - Diagonalize              0.107          54.92
 - Density                  0.002           1.23
 - Charges                  0.002           0.92
Forces autograd             0.158          34.10
------------------------------------------------
Sum                         0.458          98.75
Total                       0.464 